In [5]:
!pip install pyspark

In [6]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from google.colab import output
output.serve_kernel_port_as_window(4040, path='/jobs/index.html')

<IPython.core.display.Javascript object>

In [2]:
import time
import string
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from pyspark import SparkContext

sc = SparkContext("local", "Word_Count_App")


def get_tokens_list(data):
	if data.strip():
		data = data.translate(str.maketrans("", "", string.punctuation))

	return [token.lower() for token in data.split()]


def word_frequency(combined_data):
	tokens = combined_data.flatMap(lambda data: get_tokens_list(data))

	stop_words = set(stopwords.words("english"))
	tokens_without_stopwords = tokens.filter(lambda token: token not in stop_words)

	tokens_count = tokens_without_stopwords.map(lambda token: (token, 1))
	final_tokens_count = tokens_count.reduceByKey(lambda i, j: i + j)

	sorted_tokens_count = final_tokens_count.sortBy(lambda items: items[1], ascending=False)
	sorted_tokens_count = sorted_tokens_count.filter(lambda x: x[0] != '')

	return sorted_tokens_count


def main() -> None:
	path_1 = "/content/drive/MyDrive/Colab Notebooks/dataset/DRACULA.txt"
	path_2 = "/content/drive/MyDrive/Colab Notebooks/dataset/Metamorphosis.txt"
	output_path = "/content/drive/MyDrive/Colab Notebooks/dataset/extended_wc_output/"
	file1_data = sc.textFile(path_1)
	file2_data = sc.textFile(path_2)
	combined_data = file1_data.union(file2_data)

	sorted_word_count_frequency = word_frequency(combined_data)
	sorted_word_count_frequency.coalesce(1).saveAsTextFile(output_path)
	time.sleep(180)
	sc.stop()


main()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
